# Automated ML

Import Dependencies. 

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.run import Run
import pandas as pandas
from azureml.data.datapath import DataPath
from azureml.train.automl import AutoMLConfig
import joblib
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


## Dataset

### Overview
This experiment with AutoML uses a dataset from Kaggle regarding heart failures with 12 features to predict whether an event of death occurs or not. Therefore, this is a classification prediction task.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EYTT2D5J8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
ds = Dataset.get_by_name(ws, 'dataset')

In [4]:
df = ds.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
train_data, test_data = ds.random_split(0.7)

## AutoML Configuration

In the AutoML configuration the following parameters are applied:
- experiment_timeout_minutes:30, The number of minutes the experiment should stay running.
- primary_metric:'accuracy', The primary metric parameter for optimization.
- n_cross_validations:5, As the dataset is relatively small, a low number of number of cross validations is applied.
- max_concurrent_iterations: 4, To get a dedicated cluster per experiment, the max_concurrent_iterations is set at four, to manage the child runs and when they can be performed. Four max_concurrent_iterations is chosen, as the number of nodes in the cluster is also four.
- training_data=train_data, The train_data is part of the dataset uploaded for this experiment.
- label_column_name='DEATH_EVENT', The target column is death_event, as we want to predict whether a patient passes away.
- featurization='auto', Automatic guardrails and featurization steps part of preprocessing.
- compute_target=cpu_cluster, The ML model is trained on a cluster of Azure virtual machines from Azure Machine Learning Managed Compute. 'STANDARD_D2_V2' with a maximum of 4 nodes in a cpu_cluster is used as compute_target.
- task = 'classification', As the goal is to predict whether someone dies or not, this experiment becomes a classification task type.

In [6]:
cpu_cluster_name = "computecluster"

# Verify if the clusters already exists
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing target, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output = True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
#Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "primary_metric":'accuracy',
    "n_cross_validations":5,
    "max_concurrent_iterations": 4}

# Put your automl config here
automl_config = AutoMLConfig(training_data=train_data,
label_column_name='DEATH_EVENT',
featurization='auto',
    compute_target=cpu_cluster,
    task = 'classification',
    **automl_settings)

In [8]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on computecluster with default configuration
Running on remote compute: computecluster
Parent Run ID: AutoML_307671fa-1a32-41a0-bdab-44667d03963f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputa

## Run Details

`RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_307671fa-1a32-41a0-bdab-44667d03963f',
 'target': 'computecluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-22T07:52:40.274144Z',
 'endTimeUtc': '2021-02-22T08:36:44.570221Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'computecluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-139304","workspace_name":"quick-starts-ws-139304","region":"southcentralus","compute_target":"computecluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,

## Best Model

The best model from the automl experiments and display all the properties of the model.



In [10]:
# Retrieve and save your best automl model.
best_run, best_model = remote_run.get_output()
print(best_run)

Package:azureml-automl-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-core, training version:1.22.0, current version:1.20.0
Package:azureml-dataprep, training version:2.9.1, current version:2.7.3
Package:azureml-dataprep-native, training version:29.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.7.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-defaults, training version:1.22.0, current version:1.20.0
Package:azureml-interpret, training version:1.22.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.22.0, current version:1.20.0
Package:azureml-telemetry, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.22.0, current version:1.20.0


Run(Experiment: automl,
Id: AutoML_307671fa-1a32-41a0-bdab-44667d03963f_103,
Type: azureml.scriptrun,
Status: Completed)


In [11]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_307671fa-1a32-41a0-bdab-44667d03963f_103,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
print(best_model)


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.29105263157894734,
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=600,
           

In [14]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

joblib.dump(best_model, filename="outputs/automl-model.pkl")

automl_model = remote_run.register_model(model_name='best-automl-model', description='training context: AutoML')
#automl_model = best_run.register_model(model_name = 'best_automl_model.pkl', model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.automl.core.shared import constants

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Heart Failure Prediction',
    auth_enabled=True
)

In [20]:
from azureml.core.model import Model

model = Model(ws, 'best-automl-model')

myenv = best_run.get_environment()
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

inference_config=InferenceConfig(entry_script='score.py', environment=myenv)

service=Model.deploy(workspace=ws,
    name='aci-service',
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [21]:
#enable application insights
service.update(enable_app_insights=True)

In [22]:
print('state: ' + service.state)
print('key: ' +service.get_keys()[0])
print('swagger URI: ' +service.swagger_uri)
print('scoring URI: ' +service.scoring_uri)

state: Healthy
key: 9NXiRDaILuK5JxFtaPgdfU8dlSwdWMEy
swagger URI: http://ff6a9530-fa7f-46e4-8622-fc9f024977d1.southcentralus.azurecontainer.io/swagger.json
scoring URI: http://ff6a9530-fa7f-46e4-8622-fc9f024977d1.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
import requests
import json

#two sets of data to score

td = test_data.to_pandas_dataframe()
sample_data = td.sample(2)
y_test = sample_data['DEATH_EVENT']
sample_data.drop(['DEATH_EVENT'], inplace=True,axis=1)
x_test = sample_data
data={'data':x_test.to_dict(orient='records')}

#Convert to JSON
input_data = json.dumps(data)
print(input_data)

{"data": [{"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 80, "high_blood_pressure": 0, "platelets": 263358.03, "serum_creatinine": 1.18, "serum_sodium": 137, "sex": 0, "smoking": 0, "time": 63}, {"age": 40.0, "anaemia": 0, "creatinine_phosphokinase": 90, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 0, "platelets": 255000.0, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 212}]}


In [25]:
#set content type
headers = {'content-type': 'application/json'}

#if authorization enabled, set authorization ehader
headers['Authorization'] = f'Bearer {service.get_keys()[0]}'

#make the request and display the response
resp = requests.post(service.scoring_uri, input_data, headers=headers)
print('prediction: ', resp.text)

#print original labels
print('True Values: ', y_test.values)

prediction:  "{\"result\": [0, 0]}"
True Values:  [0 0]


In [26]:
print(service.get_logs())

2021-02-22T08:57:09,645422791+00:00 - iot-server/run 
2021-02-22T08:57:09,646525502+00:00 - gunicorn/run 
2021-02-22T08:57:09,649334383+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-22T08:57:09,661324682+00:00 - rsyslog/run 
rsyslogd

TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
service.delete()
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

